Scraper to get quarter financial reports for the last years. Gathered information contains Income statements, Balance sheets, Cashflow statements and Ratios.
Also we retrieve a list of stocks tickers evaluated by market capitalization and their historical daily prices for a predefined range of time.

In [2]:
import pandas as pd
import datetime
import requests
import yfinance as yf
import joblib
import config

from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#### Functions

In [3]:
def get_tickers_mcap_gt(n_millions):
    '''
    Get stocks' tickers and sectors with market capitalization 
    greater than n_millions USD listed on NYSE and NASDAQ
    '''
    url = config.url
    response = requests.get(url)
    df = pd.DataFrame(response.json())
    df = df[~df.sector.isna()]
    df.symbol = df.symbol.str.replace('-', '.')
    
    return df[['symbol', 'sector']]

def closest_end_of_month(date):
    # Get the last day of the current month
    current_month_end = date + pd.tseries.offsets.MonthEnd(0)
    # Get the last day of the previous month
    previous_month_end = date - pd.tseries.offsets.MonthEnd(1)
    
    # Return the closest end of the month
    if abs(current_month_end - date) <= abs(previous_month_end - date):
        return current_month_end 
    else:
        return previous_month_end

def scrape_ticker(driver):
    '''
    Scrape quarter history data for a ticker
    '''
    # Wait until the button 'Quarterly' is clickable
    quarterly_button = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Quarterly")]'))
    )
    quarterly_button.click()

    # Wait for the table to load
    table = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'table[data-test="financials"]'))
    )
    # Wait for 'Quarter Ended' header to appear
    WebDriverWait(driver, 3).until(
        EC.visibility_of_element_located((By.XPATH, '//th[contains(text(), "Quarter Ended")]'))
    )
    # Extract table headers
    headers = [header.text for header in table.find_elements(By.TAG_NAME, 'th')]

    # Extract table rows
    rows = table.find_elements(By.TAG_NAME, 'tr')

    # Extract data from each row
    data = []
    for row in rows[1:]:  # Skip the header row
        cols = row.find_elements(By.TAG_NAME, 'td')
        data.append([col.text for col in cols])

    # Create a DataFrame
    df = pd.DataFrame(data, columns=headers).set_index('Quarter Ended').T[:-1]
    df = df.rename_axis(columns='')
    df = df.reset_index(names='Date')
    df = df[~df.Date.eq('Current')]
    df.Date = pd.to_datetime(df.Date)
    df.Date = df.Date.map(closest_end_of_month)

    return df

def scrape_all_tickers(tickers, report_type='Income'):
    '''
    Scrape quarter history data for a list of tickers
    '''
    edgedriver_path = config.edgedriver_path
    service = EdgeService(edgedriver_path)
    driver = webdriver.Edge(service=service)
    base_url = config.base_url

    # Create a list of each ticker's df
    dfs_list = []
    for ticker, sector in tickers.values:
        ticker = ticker.lower()
        # Dictionary of report types urls
        urls = {
            'Income': f'{base_url}{ticker}/financials/',
            'Balance': f'{base_url}{ticker}/financials/balance-sheet/',
            'Cashflow': f'{base_url}{ticker}/financials/cash-flow-statement/',
            'Ratios': f'{base_url}{ticker}/financials/ratios/',
        }
        while True:
            try:
                # Scrape url
                driver.get(urls[report_type])

                # Check if page exists
                if "Page Not Found" in driver.page_source:
                    print(f'{ticker} not found')
                    break
                else:
                    df = scrape_ticker(driver)
                    df.insert(loc=1, column='Sector', value=sector)
                    df.insert(loc=1, column='Ticker', value=ticker.upper())
                    dfs_list.append(df)
                    break
            except:
                print(f'For {ticker} got an error')
                break
    # Close url
    driver.quit()

    # Create a common df with all tickers
    df = pd.concat(dfs_list, ignore_index=False)

    # Edit columns
    cols_to_skip = ['Date', 'Ticker', 'Sector']
    cols_to_edit = [col for col in df.columns if col not in cols_to_skip]
    
    for col in cols_to_edit:
        # Check if cells contains %
        is_procent = False
        if (df[col].astype(str).apply(lambda x: '%' in x)).any():
            is_procent = True
        
        # Correct formats
        df[col] = df[col].str.replace('%', '')
        df[col] = df[col].str.replace(',', '')
        df[col] = pd.to_numeric(df[col], downcast='float', errors='coerce')

        # Rename columns that contains %
        if is_procent:
            df = df.rename(columns={col: col + ' %'})
    
    return df

def update_data(report_type, month_to_restore, month_to_save, tickers):
    '''
    Update financial dataframes with newly scraped data
    '''
    # Scrape data
    df = scrape_all_tickers(tickers, report_type=report_type)

    # Restore previous data
    df_restored = pd.read_parquet(
        f'100m_quarter_{report_type.lower()}_04_2019_to_{month_to_restore}.parquet'
        )
    # Keep just the new data that wasn't in previous data
    restored_max_date = df_restored.Date.max()
    df = df[df.Date.gt(restored_max_date)]

    # Concatenate and save all the data
    df = pd.concat([df_restored, df])
    df = df.sort_values(by=['Ticker', 'Date'], ascending=[True, False])
    df.to_parquet(f'100m_quarter_{report_type.lower()}_04_2019_to_{month_to_save}.parquet')
    df.to_csv(f'100m_quarter_{report_type.lower()}_04_2019_to_{month_to_save}.parquet')

    return df

def get_prices(tickers, start_date):
    '''
    Download prices history for a list of tickers 
    '''
    tickers = tickers.Ticker.str.replace('.', '-')
    prices_dfs = []
    for ticker in tickers:
        try:
            prices = yf.download(ticker, start=start_date)
            prices = prices.rename(columns={'Close': 'Price'})
            prices['Ticker'] = ticker
            prices = prices.sort_index(ascending=False)
            prices = prices[['Price', 'Ticker']]
            prices_dfs.append(prices)
        except:
            print(f'{ticker} not found')
    
    prices = pd.concat(prices_dfs).reset_index()
    prices.Ticker = prices.Ticker.str.replace('-', '.')

    return prices

def update_price(tickers, month_to_restore, start_date):
    '''
    Update prices dataframe
    '''
    restore_prices = pd.read_parquet(f'prices_01_2018_to_{month_to_restore}.parquet')
    new_prices = get_prices(tickers, start_date)
    prices = pd.concat([restore_prices, new_prices])
    prices = prices.drop_duplicates()

    return prices.sort_values(['Ticker', 'Date'], ascending=[True, False])

#### Scraper

In [4]:
# Define scraper parameters
report_type = 'Income'
month_to_restore = '05_2024'
prices_start_date = '2019-01-01'
month_to_save = datetime.date.today().strftime('%m_%Y')
# tickers_gt_100m = get_tickers_mcap_gt(n_millions=100)
tickers_gt_100m = pd.read_parquet('tickers_gt_100m.parquet')

In [5]:
# Update financial data
df = update_data(report_type, month_to_restore, month_to_save, tickers_gt_100m)

# Save financial data
df.to_parquet(f'100m_quarter_{report_type}_04_2019_to_{month_to_save}.parquet')
df.to_csv(f'100m_quarter_{report_type}_04_2019_to_{month_to_save}.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 69003 entries, 0 to 69475
Data columns (total 42 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Date                          69003 non-null  datetime64[ns]
 1   Ticker                        69003 non-null  object        
 2   Sector                        69003 non-null  object        
 3   Revenue                       66388 non-null  float32       
 4   Revenue Growth (YoY) %        61166 non-null  float32       
 5   Cost of Revenue               56184 non-null  float32       
 6   Gross Profit                  66416 non-null  float32       
 7   Selling, General & Admin      67325 non-null  float32       
 8   Research & Development        27820 non-null  float32       
 9   Operating Expenses            68745 non-null  float32       
 10  Operating Income              68845 non-null  float32       
 11  Interest Expense / Income     545

,Date,Ticker,Sector,Revenue,Revenue Growth (YoY) %,Cost of Revenue,Gross Profit,"Selling, General & Admin",Research & Development,Operating Expenses,...,EBITDA,EBITDA Margin %,Depreciation & Amortization,EBIT,EBIT Margin %,Other Operating Expenses,Interest Income,Interest Expense,Preferred Dividends,Net Income Common
0,2024-03-31,AAPL,Technology,90753.0,-4.31,48482.0,42271.0,6468.0,7903.0,14371.0,...,30894.0,34.040001,2836.0,28058.0,30.920000,NaN,NaN,NaN,NaN,NaN
1,2023-12-31,AAPL,Technology,119575.0,2.07,64720.0,54855.0,6786.0,7696.0,14482.0,...,43171.0,36.099998,2848.0,40323.0,33.720001,NaN,NaN,NaN,NaN,NaN
2,2023-09-30,AAPL,Technology,89498.0,-0.72,49071.0,40427.0,6151.0,7307.0,13458.0,...,30653.0,34.250000,2653.0,28000.0,31.290001,NaN,NaN,NaN,NaN,NaN
3,2023-06-30,AAPL,Technology,81797.0,-1.40,45384.0,36413.0,5973.0,7442.0,13415.0,...,26783.0,32.740002,3052.0,23731.0,29.010000,NaN,NaN,NaN,NaN,NaN
4,2023-03-31,AAPL,Technology,94836.0,-2.51,52860.0,41976.0,6201.0,7457.0,13658.0,...,31280.0,32.980000,2898.0,28382.0,29.930000,NaN,NaN,NaN,NaN,NaN


In [6]:
# Update prices 
prices = update_price(tickers_gt_100m, month_to_restore, prices_start_date)

# Save prices
prices.to_parquet(f'prices_01_2018_to_{month_to_save}.parquet')

prices.info()
prices.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4510783 entries, 0 to 4510782
Data columns (total 3 columns):
 #   Column  Dtype         
---  ------  -----         
 0   Date    datetime64[ns]
 1   Price   float64       
 2   Ticker  object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 103.2+ MB


,Date,Price,Ticker
0,2024-06-18,214.289993,AAPL
1,2024-06-17,216.669998,AAPL
2,2024-06-14,212.490005,AAPL
3,2024-06-13,214.240005,AAPL
4,2024-06-12,213.070007,AAPL
